In [4]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

plt.style.use('Solarize_Light2')

- **Load data**

In [5]:
WPSFD49207_df = pd.read_csv('../data/WPSFD49207.csv')

WPSFD49207_df['DATE'] = pd.to_datetime(WPSFD49207_df['DATE'])
WPSFD49207_df.set_index('DATE', inplace=True)

WPSFD49207_df['^ %'] = WPSFD49207_df['WPSFD49207'].pct_change()

WPSFD49207_df.tail()

,WPSFD49207,^ %
DATE,,
2025-07-01,262.510,0.004400
2025-08-01,262.787,0.001055
2025-09-01,265.516,0.010385
2025-10-01,263.829,-0.006354
2025-11-01,266.904,0.011655


- **Average change**

In [6]:
print(f"Average change is {WPSFD49207_df['^ %'].mean() * 100}%")

Average change is 0.24916456860862932%


- **Overview on change**

In [7]:
positive_filter = WPSFD49207_df['^ %'] > 0
negative_filter = WPSFD49207_df['^ %'] < 0
zero_filter = WPSFD49207_df['^ %'] == 0

# count
total_items = len(WPSFD49207_df)

# average change
avg_change = [
    WPSFD49207_df[positive_filter]['^ %'].mean(),
    WPSFD49207_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    WPSFD49207_df[positive_filter]['^ %'].count(),
    WPSFD49207_df[negative_filter]['^ %'].count(),
    WPSFD49207_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [100 * frequency[i]/total_items for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [x * 100 for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.579544,590.000000,62.5,0.362215
Av Neg,-0.467112,229.000000,24.258475,-0.113314
Zero,0,124.000000,13.135593,0.0
Ratio P/N,,2.576419,,


- **Stats**

In [8]:
from scipy.stats import describe

# Display the results

stats = describe(WPSFD49207_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            WPSFD49207_df['^ %'].median() * 100,
            WPSFD49207_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,943
Min %,-3.028603
Max %,3.524229
Mean %,0.249165
Median %,0.258532
Mode %,0.0
Variance,0.000038
Skewness,0.020954
Kurtosis,3.848667


- **Data preview**

In [9]:
# Define bins
bins = [0.005 * i for i in range(-2, 5)]
bins.append(stats.minmax[1])
bins.insert(0, stats.minmax[0])
bin_labels = [f"{round(bins[i] * 100, 2)}% to {round(bins[i+1] * 100, 2)}%" for i in range(len(bins) - 1)]
bin_labels[0] = f"Less than {bins[1] * 100}%"
bin_labels[-1] = f"Greater than {bins[-2] * 100}%"

# Assign data to bins
binned = pd.cut(WPSFD49207_df['^ %'], bins=bins, labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
Less than -1.0%,24,2.545069,2.545069
-1.0% to -0.5%,41,4.347826,6.892895
-0.5% to 0.0%,288,30.540827,37.433722
0.0% to 0.5%,325,34.464475,71.898197
0.5% to 1.0%,178,18.875928,90.774125
1.0% to 1.5%,59,6.256628,97.030753
1.5% to 2.0%,17,1.802757,98.833510
Greater than 2.0%,11,1.166490,100.000000


In [10]:
fig = px.bar(occurrence_frequencies, x=occurrence_frequencies.index, y='Probability %')

fig = px.bar(
    occurrence_frequencies,
    x=occurrence_frequencies.index,
    y='Probability %',
)

fig.update_layout(
    title=dict(text='WPSFD49207 % change'),
    width=800,
    height=600,
    yaxis=dict(title="Frequency"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()

- WPSFD49207 % change

In [11]:
fig = go.Figure()

# Reference line
fig.add_trace(go.Scatter(
    x=WPSFD49207_df.index,
    y=[100 * stats.mean] * (stats.nobs + 1),
    name='Average % change',
    line=dict(
        width=2,
        color="#F2C14E",   # muted gold
        dash="dash"
    ),
    hoverinfo="skip",
    # showlegend=False,
))

# Bars
fig.add_trace(go.Bar(
    x=WPSFD49207_df.index,
    y=WPSFD49207_df['^ %'] * 100,
    name='WPSFD49207',
    marker=dict(
        color="#4C78A8",  # professional blue
        line=dict(color="#4C78A8", width=0)
    ),
    width=0.5,
))

fig.update_layout(
    title=dict(
        text='WPSFD49207 % change',
        x=0.02,
        font=dict(size=20)
    ),
    plot_bgcolor="#0E1117",
    paper_bgcolor="#0E1117",
    font=dict(color="#E6E6E6", size=14),
    width=1400,
    height=700,
    yaxis=dict(
        title="%",
        exponentformat="none",
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    hovermode="x unified",
)

fig.show()


- **WPSFD49207 data plot**

In [12]:
fig = px.line(
    WPSFD49207_df,
    x=WPSFD49207_df.index,
    y="WPSFD49207",
    markers=True
)

fig.update_layout(
    title=dict(text='Producer Price Index by Commodity: Final Demand: Finished Goods'),
    width=1000,
    height=600,
    yaxis=dict(title="WPSFD49207"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()